<a href="https://colab.research.google.com/github/monjjjjj/orchid_classification/blob/main/orchid_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.python import keras
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2 
from tensorflow import keras
from google.colab import drive
from google.colab.patches import cv2_imshow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras import Sequential
from keras import initializers, optimizers
from keras.layers import Dense, Flatten, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from keras.applications.resnet import ResNet50
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

In [ ]:
# google drive mounted
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# read data
labels = []
images = []
paths = []

train_dir = "/content/drive/MyDrive/orchid_classification/training/"
label_dir = "/content/drive/MyDrive/orchid_classification/label/label.csv"

#BATCH_SIZE = 32
#IMG_SIZE = (224, 224)

#train_dataset = image_dataset_from_directory(train_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)



In [ ]:
'''
for num in range(len(df)):
    paths.append(os.path.join('train_dir', df['filename'][:len(df)][num]))
    labels.append(df['category'][:len(df)][num])

#print(len(paths))
#print(paths[0:5])
'''

"\nfor num in range(len(df)):\n    paths.append(os.path.join('train_dir', df['filename'][:len(df)][num]))\n    labels.append(df['category'][:len(df)][num])\n\n#print(len(paths))\n#print(paths[0:5])\n"

In [ ]:
#read data by yi han
def load_data():
  x_train = []
  y_train = []
  df = pd.read_csv(label_dir, index_col = "filename")
  for p in os.listdir(train_dir):
    if p != "label.csv":
      path = train_dir + p
      image = cv2.imread(path)
      image = image.astype("float") / 255.0
      image = cv2.resize(image, (224,224), interpolation = cv2.INTER_AREA)
      x_train.append(np.array(image))
      y_train.append(df.loc[str(p),"category"])
  return x_train,y_train



In [ ]:
x_train, y_train = load_data()

In [ ]:
# debug
#print(x_train[1:5])
#print(y_train[1:5])
#print(x_train[1].shape)
#print(y_train[1].shape)

In [ ]:
'''
# autotune
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
'''

In [ ]:
#check all the files are images
'''
def start(Path):
    filelist = os.listdir(Path)
    for file in filelist:
        print(file)
        img = Image.open(Path+ file).convert('RGB')
    #     # print(img)
        img.save(Path + file)
    print('Done!')
if __name__ == '__main__':
    start('/content/drive/MyDrive/orchid_classification/training/')
'''

In [ ]:
# read data
'''
def read_image(paths):
  for path in paths:
    image = cv2.imread(path, cv2.COLOR_GRAY2BGR)
    image = image.astype("float") / 255.0
    image = cv2.resize(image,(224,224))
    images.append(image)

read_image('/content/drive/MyDrive/orchid_classification/training/')
images = np.asarray(images)
images = images.reshape((-1, 224, 224, 3))
labels = to_categorical(labels, 219)
print(1, labels)
'''

In [ ]:
'''
# data augmentation
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])
'''

In [ ]:
#data augmentation
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)


In [ ]:
# build model
def build_model():
  model = Sequential()
  model.add(ResNet50(include_top=False, weights='imagenet', input_tensor=None, pooling='avg', input_shape=(224, 224, 3)))
  model.add(Flatten())
  model.add(BatchNormalization())

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  print(model.summary())
  keras.utils.plot_model(model, show_shapes=True, dpi=64, to_file='model2.png')

  return model

In [ ]:
#training
model = build_model()

train_history = model.fit(x=x_train, y=y_train, batch_size=32, epochs=20, verbose=2)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 batch_normalization (BatchN  (None, 2048)             8192      
 ormalization)                                                   
                                                                 
Total params: 23,595,904
Trainable params: 23,538,688
Non-trainable params: 57,216
_________________________________________________________________
None


ValueError: ignored

In [ ]:
print(train_history.history.keys())

In [ ]:
def show_train_history(train_history,train,validation):
  
  if train == 'accuracy':
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Model Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
  else:
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Model Loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')

  plt.legend(['train','validation'],loc='upper left')
  plt.show()

In [ ]:
show_train_history(train_history,'accuracy','val_accuracy')
show_train_history(train_history,'loss','val_loss')

In [ ]:
# summarize history for accuracy
plt.plot(train_history.history['accuracy'])
plt.plot(train_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left') 
plt.show()